# Chapter 8: Time Series Analysis

In finance and economics, a huge amount of our data is in the format of time-series,
such as stock prices and Gross Domestic Products (GDP). From Chapter 4, Sources
of Data, it is shown that from Yahoo!Finance, we could download daily, weekly, and
monthly historical price time-series. From Federal Reserve Bank's Economics Data
Library (FRED), we could retrieve many historical time-series such as GDP. For
time-series, there exist many issues, such as how to estimate returns from historical
price data, how to merge datasets with the same or different frequencies, seasonality,
and detect auto-correlation. Understanding those properties is vitally important for
our knowledge development.

In this chapter, the following topics will be covered:
• Introduction to time-series analysis

• Design a good date variable, and merging different datasets by date

• Normal distribution and normality test

• Term structure of interest rates, 52-week high, and low trading strategy

• Return estimation and converting daily returns to monthly or annual returns

• T-test, F-test, and Durbin-Watson test for autocorrelation

• Fama-MacBeth regression

• Roll (1984) spread, Amihud's (2002) illiquidity, and Pastor and Stambaugh's
(2003) liquidity measure

• January effect and weekday effect

• Retrieving high-frequency data from Google Finance and Prof. Hasbrouck's
TORQ database (Trade, Order, Report, and Quotation)

• Introduction to CRSP (Center for Research in Security Prices) database

## Introduction to Time Series

Most finance data is in the format of time-series, see the following several examples.
The first one shows how to download historical, daily stock price data from
Yahoo!Finance for a given ticker's beginning and ending dates:

In [9]:
import yfinance as yf
x = yf.download('IBM', start="2016-01-01", end="2016-01-21", rounding=True)
print(x)

[*********************100%***********************]  1 of 1 completed
              Open    High     Low   Close  Adj Close    Volume
Date                                                           
2016-01-04  129.64  129.99  128.34  129.97      89.53   5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47   4105341
2016-01-06  128.47  129.62  127.74  129.23      89.02   4509201
2016-01-07  127.82  129.08  126.61  127.02      87.50   7348987
2016-01-08  127.32  127.93  125.54  125.84      86.69   4981784
2016-01-11  126.01  127.93  125.97  127.37      87.74   5203222
2016-01-12  127.77  127.90  125.36  127.06      87.53   5312320
2016-01-13  127.63  128.37  125.33  125.40      86.39   4916305
2016-01-14  126.13  127.91  125.45  127.07      87.53   5972242
2016-01-15  124.29  125.12  123.20  124.31      85.64   9422891
2016-01-19  124.39  126.29  122.43  122.48      84.37  10438662
2016-01-20  113.25  118.54  112.81  116.50      80.25  16901059


Let's see if webreader still works. EDIT: IT doesn't for yahoo. Not an issue. Now let's grab some GDP data.

In [4]:
import pandas_datareader.data as web
import datetime
begdate = datetime.datetime(1900, 1, 1)
enddate = datetime.datetime(2017, 1, 27)
y= web.DataReader("GDP", "fred", begdate,enddate)
print(x.head(2))
print(x.tail(3))

                GDP
DATE               
1947-01-01  243.164
1947-04-01  245.968
                  GDP
DATE                 
2016-07-01  18892.639
2016-10-01  19089.379
2017-01-01  19280.084


GDP data works fine when read through the pandas_datareader module.

## Merging datasets based on a date variable

To make our time-series more manageable, it is a great idea to generate a date
variable. When talking about such a variable, readers could think about year (YYYY),
year and month (YYYYMM) or year, month, and day (YYYYMMDD). For just the
year, month, and day combination, we could have many forms. Using January 20,
2017 as an example, we could have 2017-1-20, 1/20/2017, 20Jan2017, 20-1-2017, and
the like. In a sense, a true date variable, in our mind, could be easily manipulated.
Usually, the true date variable takes a form of year-month-day or other forms of its
variants. Assume the date variable has a value of 2000-12-31. After adding one day to
its value, the result should be 2001-1-1.

We could easily use the pandas.date_range() function to generate our time-series;
refer to the following example:

In [10]:
import pandas as pd
import numpy as np
import scipy as sp

np.random.seed(1257)
mean=0.10
std=0.2
ddate = pd.date_range('1/1/2016', periods=252)
n=len(ddate)
rets= np.random.normal(mean,std,n)
data = pd.DataFrame(rets, index=ddate,columns=['RET'])
print(data.head())

                 RET
2016-01-01  0.431031
2016-01-02  0.279193
2016-01-03  0.002549
2016-01-04  0.109546
2016-01-05  0.068252


In [11]:
print(x[0:4])

              Open    High     Low   Close  Adj Close   Volume
Date                                                          
2016-01-04  129.64  129.99  128.34  129.97      89.53  5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47  4105341
2016-01-06  128.47  129.62  127.74  129.23      89.02  4509201
2016-01-07  127.82  129.08  126.61  127.02      87.50  7348987


Here are several ways to define a date variable:

In [12]:
# For a range of dates
pd.date_range('1/1/2017',periods=252)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03',
               '2017-09-04', '2017-09-05', '2017-09-06', '2017-09-07',
               '2017-09-08', '2017-09-09'],
              dtype='datetime64[ns]', length=252, freq='D')

In [13]:
# For a single date
datetime.date(2017,1,20)

datetime.date(2017, 1, 20)

In [14]:
# For today's date
datetime.date.today()

datetime.date(2024, 2, 18)

In [20]:
# Get the current time
from datetime import datetime
datetime.now()


datetime.datetime(2024, 2, 18, 13, 59, 31, 119814)

Retrieving the year, month, and day from a date variable is used quite frequently
when dealing with time-series—see the following Python program by using the
strftime() function. The corresponding output is in the following right panel. The
format of those results of year, month, and day, is string:

In [21]:
import datetime
today=datetime.date.today()
year=today.strftime("%Y")
year2=today.strftime("%y")
month=today.strftime("%m")
day=today.strftime("%d")
print(year,month,day,year2)

2024 02 18 24


## Return Estimation

With price data, we could calculate returns. In addition, sometimes we have to
convert daily returns to weekly or monthly, or convert monthly returns to quarterly
or annual ones. Thus, understanding how to estimate returns and their conversion is
vital. Assume that we have the following four prices:

In [22]:
p=[1,1.1,0.9,1.05]

In [23]:
print(p[:-1])
print(p[1:])

[1, 1.1, 0.9]
[1.1, 0.9, 1.05]


To estimate returns, we could use the following code:

In [26]:
ret=p[1:]/p[:-1]-1
ret

# this doesn't appear to work

TypeError: unsupported operand type(s) for /: 'list' and 'list'

Alternatively we can use numpy and the pandas libraries to do the same calculation, but in dataframe form.

In [27]:
p=[1,1.1,0.9,1.05]
a=pd.DataFrame({'Price':p})
a['Ret']=a['Price'].diff()/a['Price'].shift(1)
print(a)

   Price       Ret
0   1.00       NaN
1   1.10  0.100000
2   0.90 -0.181818
3   1.05  0.166667


Let's now analyze daily price data using what we've done above.

In [69]:
x = yf.download('IBM', start="2013-01-01", end="2013-11-9", rounding=True)
ret=x['Close'].pct_change()
ret

[*********************100%***********************]  1 of 1 completed


Date
2013-01-02         NaN
2013-01-03   -0.005540
2013-01-04   -0.006535
2013-01-07   -0.004368
2013-01-08   -0.001408
                ...   
2013-11-04    0.005778
2013-11-05   -0.013404
2013-11-06    0.007528
2013-11-07    0.004495
2013-11-08   -0.000058
Name: Close, Length: 217, dtype: float64

In [70]:
x.index[0:3]

DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04'], dtype='datetime64[ns]', name='Date', freq=None)

In [71]:
x['Close'][0:3] 

Date
2013-01-02    187.72
2013-01-03    186.68
2013-01-04    185.46
Name: Close, dtype: float64

In [72]:
ret[0:3]

Date
2013-01-02         NaN
2013-01-03   -0.005540
2013-01-04   -0.006535
Name: Close, dtype: float64

## Converting daily returns to monthly ones

Sometimes, we need to convert daily returns to monthly or annual ones. Here is our
procedure. First, we estimate the daily log returns. We then take a summation of
all daily log returns within each month to find out the corresponding monthly log
returns. The final step is to convert a log monthly return to a monthly percentage
return. Assume that we have the price data of p0, p1, p2, …., p20, where p0 is the last
trading price of the last month, p1 is the first price of this month, and p20 is the last
price of this month. Thus, this month's percentage return is given as follows:

R(monthly) = (p20 - p0) / p0

LogR(monthly) = log (p20/p0)

R(monthly) = exp(LogR) -1

In [73]:
logret = pd.DataFrame(np.log(1+ x['Close'].pct_change()))
logret = logret.rename(columns={'Close':'Returns'})
yyyymm=[]
d0=x.index

logret
d0


DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07',
               '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-14', '2013-01-15',
               ...
               '2013-10-28', '2013-10-29', '2013-10-30', '2013-10-31',
               '2013-11-01', '2013-11-04', '2013-11-05', '2013-11-06',
               '2013-11-07', '2013-11-08'],
              dtype='datetime64[ns]', name='Date', length=217, freq=None)

In [78]:
#
logret['Months'] = d0.strftime("%Y-%m")
logret

retMonthly=logret.groupby(logret['Months']).sum()
print(retMonthly.head())


          Returns
Months           
2013-01  0.033628
2013-02 -0.011084
2013-03  0.060232
2013-04 -0.051779
2013-05  0.026702


## Merging datasets by date

The following program merges the daily adjusted closing price of IBM with the
daily Fama-French 3-factor time-series.

In [146]:
ticker='IBM'
begdate=datetime.date(2016,1,2)
enddate=datetime.date(2017,1,9)
x = yf.download(ticker, start = begdate, end = enddate, rounding = True)

f3factor = pd.read_csv('/opt/code/test_repo/Python-For-Finance-2nd-Edition/F-F_Research_Data_Factors_daily.CSV').dropna()
f3factor['Date'] = pd.to_numeric(f3factor['Date'], errors="coerce").astype(int).dropna()
f3factor['Date'] = pd.to_datetime(f3factor['Date'], format='%Y%m%d')
f3factor = f3factor.set_index('Date')

myName=ticker+'_adjClose'
x2= pd.DataFrame(x['Close']).rename(columns={'Close':myName})

f3factor


[*********************100%***********************]  1 of 1 completed


,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-12-22,0.21,0.64,0.09,0.021
2023-12-26,0.48,0.69,0.46,0.021
2023-12-27,0.16,0.14,0.12,0.021


In [147]:
final=pd.merge(x2,f3factor,left_index=True,right_index=True)
print(final.head())
final.head()

            IBM_adjClose  Mkt-RF   SMB   HML   RF
Date                                             
2016-01-04        129.97   -1.59 -0.87  0.52  0.0
2016-01-05        129.88    0.12 -0.19  0.01  0.0
2016-01-06        129.23   -1.35 -0.14  0.00  0.0
2016-01-07        127.02   -2.44 -0.29  0.08  0.0
2016-01-08        125.84   -1.11 -0.50 -0.03  0.0


,IBM_adjClose,Mkt-RF,SMB,HML,RF
Date,,,,,
2016-01-04,129.97,-1.59,-0.87,0.52,0.0
2016-01-05,129.88,0.12,-0.19,0.01,0.0
2016-01-06,129.23,-1.35,-0.14,0.00,0.0
2016-01-07,127.02,-2.44,-0.29,0.08,0.0
2016-01-08,125.84,-1.11,-0.50,-0.03,0.0


## Understanding the interpolation technique

Interpolation is a technique used quite frequently in finance. In the following
example, we have to replace two missing values, NaN, between 2 and 6. The
pandas.interpolate() function, for a linear interpolation, is used to fill in the
two missing values:

In [148]:
import pandas as pd
import numpy as np
nn=np.nan
x=pd.Series([1,2,nn,nn,6])
print(x.interpolate())

0    1.000000
1    2.000000
2    3.333333
3    4.666667
4    6.000000
dtype: float64


## Merging data with different frequencies

The following Python program merges two datasets: US Gross Domestic Product
(GDP) data with a quarterly frequency and ffMonthly, http://canisius.
edu/~yany/python/ffMonthly.pkl with a monthly frequency.


The interpolation methodology discussed previously is applied to the missing
months in terms of GDP data. The ffMonthly dataset is assumed to be saved
in the c:/temp/ directory:

In [187]:
#get Fama French factors
import getFamaFrenchFactors as gff

ff3 = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3 = ff3.rename(columns={'date_ff_factors':'Date'})
ff3['Date'] = ff3['Date'].dt.strftime('%Y-%m')
ff3 = ff3.set_index('Date')
ff3



,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-08,-0.0239,-0.0316,-0.0106,0.0045
2023-09,-0.0524,-0.0251,0.0152,0.0043
2023-10,-0.0319,-0.0387,0.0019,0.0047


In [196]:
begdate = datetime.datetime(1900, 1, 1)
enddate = datetime.datetime(2024, 1, 27)
GDP = web.DataReader("GDP", "fred", begdate, enddate)
GDP.index = GDP.index.strftime('%Y-%m')
GDP

,GDP
DATE,
1947-01,243.164
1947-04,245.968
1947-07,249.585
1947-10,259.745
1948-01,265.742
...,...
2022-10,26408.405
2023-01,26813.601
2023-04,27063.012


In [197]:
final=pd.merge(ff3,GDP,left_index=True,right_index=True,how='left')
tt=final['GDP']
GDP2=pd.Series(tt).interpolate()
final['GDP2']=GDP2

In [198]:
final.tail(20)

,Mkt-RF,SMB,HML,RF,GDP,GDP2
Date,,,,,,
2022-05,-0.0034,-0.0183,0.0839,0.0003,NaN,25694.395000
2022-06,-0.0843,0.0210,-0.0597,0.0006,NaN,25844.517000
2022-07,0.0957,0.0281,-0.0405,0.0008,25994.639,25994.639000
2022-08,-0.0377,0.0140,0.0029,0.0019,NaN,26132.561000
2022-09,-0.0935,-0.0081,0.0005,0.0019,NaN,26270.483000
2022-10,0.0783,0.0006,0.0801,0.0023,26408.405,26408.405000
2022-11,0.0460,-0.0352,0.0138,0.0029,NaN,26543.470333
2022-12,-0.0641,-0.0069,0.0137,0.0033,NaN,26678.535667
2023-01,0.0665,0.0501,-0.0401,0.0035,26813.601,26813.601000


## Tests of normality

In finance, knowledge about normal distribution is very important for two reasons.
First, stock returns are assumed to follow a normal distribution. Second, the error
terms from a good econometric model should follow a normal distribution with a
zero mean. However, in the real world, this might not be true for stocks. On the other
hand, whether stocks or portfolios follow a normal distribution could be tested by
various so-called normality tests. The Shapiro-Wilk test is one of them. For the first
example, random numbers are drawn from a normal distribution. As a consequence,
the test should confirm that those observations follow a normal distribution:

In [207]:
from scipy import stats
np.random.seed(12345)
mean=0.1
std=0.2
n=5000
ret=np.random.normal(loc=0,scale=std,size=n)
print('W-test, and P-value')
print(stats.shapiro(ret))

W-test, and P-value
ShapiroResult(statistic=0.9995947480201721, pvalue=0.40313461422920227)


Assume that our confidence level is 95%, that is, alpha=0.05. The first value of the
result is the test statistic, and the second one is its corresponding P-value. Since
the P-value is so big, much bigger than 0.05, we accept the null hypothesis that the
returns follow a normal distribution. For the second example, random numbers are
drawn from a uniform distribution:

In [208]:
np.random.seed(12345)
n=5000
ret=np.random.uniform(size=n)
print('W-test, and P-value')
print(stats.shapiro(ret))

W-test, and P-value
ShapiroResult(statistic=0.9537610411643982, pvalue=4.076294275851805e-37)


Since the P-value is close to zero, we reject the null hypothesis. In other words, those
observations do not follow a normal distribution. The third example verifies whether
IBM's returns follow a normal distribution. The last five year's daily data from
Yahoo! Finance is used for the test. The last five year's daily data from
Yahoo! Finance is used for the test. The null hypothesis is that IBM's daily returns are
drawn from a normal distribution:

In [218]:
x = yf.download('IBM', start="2019-01-01", end="2024-01-21", rounding=True)
ret = x['Close'].pct_change().dropna()

print("Ticker =",ticker, "W-test, and P-value")
print(stats.shapiro(ret))

[*********************100%***********************]  1 of 1 completed
Ticker = IBM W-test, and P-value
ShapiroResult(statistic=0.8869530558586121, pvalue=1.762584302639909e-29)


Since this P-value is so close to zero, we reject the null hypothesis. In other
words, we conclude that IBM's daily returns do not follow a normal distribution.
For a normality test, we could also apply the Anderson-Darling test, which is a
modification of the Kolmogorov-Smirnov test, to verify whether the observations
follow a particular distribution. See the following code:

In [219]:
print( stats.anderson(ret) )

AndersonResult(statistic=26.0722603874824, critical_values=array([0.574, 0.654, 0.785, 0.915, 1.089]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.0004869297522596514, scale=0.016585637480049348)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


Here, we have three sets of values: the Anderson-Darling test statistic, a set of critical
values, and a set of corresponding confidence levels, such as 15 percent, 10 percent,
5 percent, 2.5 percent, and 1 percent, as shown in the previous output. If we choose
a 1 percent confidence level—the last value of the third set—the critical value is
1.089, the last value of the second set. Since our testing statistic is 26, which is
much higher than the critical value of 1.089, we reject the null hypothesis. Thus, our
Anderson-Darling test leads to the same conclusion as our Shapiro-Wilk test. One
of the beauties of the scipy.stats.anderson() test is that we can test for other
distributions. After applying the help() function, we would get the following list. 

In [220]:
help(stats.anderson)

Help on function anderson in module scipy.stats._morestats:

anderson(x, dist='norm')
    Anderson-Darling test for data coming from a particular distribution.
    
    The Anderson-Darling test tests the null hypothesis that a sample is
    drawn from a population that follows a particular distribution.
    For the Anderson-Darling test, the critical values depend on
    which distribution is being tested against.  This function works
    for normal, exponential, logistic, or Gumbel (Extreme Value
    Type I) distributions.
    
    Parameters
    ----------
    x : array_like
        Array of sample data.
    dist : {'norm', 'expon', 'logistic', 'gumbel', 'gumbel_l', 'gumbel_r', 'extreme1'}, optional
        The type of distribution to test against.  The default is 'norm'.
        The names 'extreme1', 'gumbel_l' and 'gumbel' are synonyms for the
        same distribution.
    
    Returns
    -------
    result : AndersonResult
        An object with the following attributes:
    
 